****************************************************************

# Data exploration

In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


### Quick-note on project directory

The main root dir `~/3dcorrection` is structured as follow:
* `data/` contains raw and preprocessed data. 
    * `raw/` is actually a symbolic link to the same repo for all candidates, DO NOT TOUCH IT!
    * `processed/` will be created when data is preprocessed and will contain all transformed data
* 

In [1]:
import os

root_path = os.path.join('/', 'root', 'bootcamps')

data_path = os.path.join(root_path, 'data')
cache_path = os.path.join(data_path, 'cache')
raw_data_path = os.path.join(data_path, 'raw')
processed_data_path = os.path.join(data_path, 'processed')

### The 3D Correction Use-Case

The European Centre for Medium-range Weather Forecasts (ECMWF) has developed a series of model giving the current best accurate parametrization scheme available—among those, SPARTACUS delivers **radiation** prediction over the globe. Because it is demanding in computations, a simpler, degraded model called TRIPLECLOUD is developed to satisfy the production environment constraints. 

Like most climate models, to leverage hardware acceleration, the choice is made to split the globe in blocks—this has the immediate consequence of losing the spatial correlation for a gain in parallelization. 

The unit block is a column that express values throughout the vertical dimension over a set of levels. Each level is

Now let's load the raw data we'll be using throughout this hands-on. Take a look at the [source notebook](https://git.ecmwf.int/projects/MLFET/repos/maelstrom-radiation/browse/climetlab_maelstrom_radiation/radiation.py) for a more info on the variables.

In [2]:
import climetlab as cml
import dask
import dask.array as da
from glob import glob
import numpy as np
import os.path as osp
import xarray as xr

import config

cml.settings.set("cache-directory", cache_path)

cmlds = cml.load_dataset(
    'maelstrom-radiation', 
    dataset='3dcorrection', 
    raw_inputs=False, 
    timestep=list(range(0, 3501, config.params['timestep'])), 
    minimal_outputs=False,
    patch=list(range(0, 16, 1)),
    hr_units='K d-1',
)

xr_array = cmlds.to_xarray()
xr_array

By downloading data from this dataset, you agree to the terms and conditions defined at https://apps.ecmwf.int/datasets/licences/general/ If you do not agree with such terms, do not download the data. 


<xarray.Dataset>
Dimensions:       (column: 1085440, sca_variable: 17, level: 137,
                   col_variable: 27, half_level: 138, hl_variable: 2,
                   p_variable: 1, level_interface: 136, inter_variable: 1)
Dimensions without coordinates: column, sca_variable, level, col_variable,
                                half_level, hl_variable, p_variable,
                                level_interface, inter_variable
Data variables: (12/13)
    sca_inputs    (column, sca_variable) float32 dask.array<chunksize=(16960, 1), meta=np.ndarray>
    col_inputs    (column, level, col_variable) float32 dask.array<chunksize=(16960, 137, 1), meta=np.ndarray>
    hl_inputs     (column, half_level, hl_variable) float32 dask.array<chunksize=(16960, 138, 1), meta=np.ndarray>
    pressure_hl   (column, half_level, p_variable) float32 dask.array<chunksize=(16960, 138, 1), meta=np.ndarray>
    inter_inputs  (column, level_interface, inter_variable) float32 dask.array<chunksize=(16960, 136, 1), meta=np.ndarray>
    lat           (column) float32 dask.array<chunksize=(16960,), meta=np.ndarray>
    ...            ...
    flux_dn_sw    (column, half_level) float32 dask.array<chunksize=(16960, 138), meta=np.ndarray>
    flux_up_sw    (column, half_level) float32 dask.array<chunksize=(16960, 138), meta=np.ndarray>
    flux_dn_lw    (column, half_level) float32 dask.array<chunksize=(16960, 138), meta=np.ndarray>
    flux_up_lw    (column, half_level) float32 dask.array<chunksize=(16960, 138), meta=np.ndarray>
    hr_sw         (column, level) float32 dask.array<chunksize=(16960, 137), meta=np.ndarray>
    hr_lw         (column, level) float32 dask.array<chunksize=(16960, 137), meta=np.ndarray>

The returned object is a ClimateLab dataset Xarray Dataset

Let's check the content of the downloaded file

most operations are computed lazily in dask/xarray when needed and if possible on every chunk, treated and seen 'as if' it was a continuous array

In [3]:
xr_array.sca_inputs

<xarray.DataArray 'sca_inputs' (column: 1085440, sca_variable: 17)>
dask.array<concatenate, shape=(1085440, 17), dtype=float32, chunksize=(16960, 6), chunktype=numpy.ndarray>
Dimensions without coordinates: column, sca_variable
Attributes:
    long_name:  Skin_temperature
    units:      K

In [4]:
features = [
    'sca_inputs',
    'col_inputs',
    'hl_inputs',
    'inter_inputs',
    'flux_dn_sw',
    'flux_up_sw',
    'flux_dn_lw',
    'flux_up_lw',
]

for feat in features:
    print(f'{feat}: {xr_array[feat].data}')

sca_inputs: dask.array<concatenate, shape=(1085440, 17), dtype=float32, chunksize=(16960, 6), chunktype=numpy.ndarray>
col_inputs: dask.array<concatenate, shape=(1085440, 137, 27), dtype=float32, chunksize=(16960, 137, 12), chunktype=numpy.ndarray>
hl_inputs: dask.array<concatenate, shape=(1085440, 138, 2), dtype=float32, chunksize=(16960, 138, 1), chunktype=numpy.ndarray>
inter_inputs: dask.array<transpose, shape=(1085440, 136, 1), dtype=float32, chunksize=(16960, 136, 1), chunktype=numpy.ndarray>
flux_dn_sw: dask.array<concatenate, shape=(1085440, 138), dtype=float32, chunksize=(16960, 138), chunktype=numpy.ndarray>
flux_up_sw: dask.array<concatenate, shape=(1085440, 138), dtype=float32, chunksize=(16960, 138), chunktype=numpy.ndarray>
flux_dn_lw: dask.array<concatenate, shape=(1085440, 138), dtype=float32, chunksize=(16960, 138), chunktype=numpy.ndarray>
flux_up_lw: dask.array<concatenate, shape=(1085440, 138), dtype=float32, chunksize=(16960, 138), chunktype=numpy.ndarray>


In [19]:
dataset_size = xr_array.dims['column']
num_shards = 53 * 2 ** 3
shard_size = dataset_size // num_shards

data = {}
# all this is lazy
for feat in features:
    array = xr_array[feat].data
    array = da.rechunk(array, chunks=(shard_size, *array.shape[1:]))
    print(type(array))
    data.update({feat: array})
    print(f'{feat}: {array}')

<class 'dask.array.core.Array'>
sca_inputs: dask.array<rechunk-merge, shape=(1085440, 17), dtype=float32, chunksize=(2560, 17), chunktype=numpy.ndarray>
<class 'dask.array.core.Array'>
col_inputs: dask.array<rechunk-merge, shape=(1085440, 137, 27), dtype=float32, chunksize=(2560, 137, 27), chunktype=numpy.ndarray>
<class 'dask.array.core.Array'>
hl_inputs: dask.array<rechunk-merge, shape=(1085440, 138, 2), dtype=float32, chunksize=(2560, 138, 2), chunktype=numpy.ndarray>
<class 'dask.array.core.Array'>
inter_inputs: dask.array<rechunk-merge, shape=(1085440, 136, 1), dtype=float32, chunksize=(2560, 136, 1), chunktype=numpy.ndarray>
<class 'dask.array.core.Array'>
flux_dn_sw: dask.array<rechunk-merge, shape=(1085440, 138), dtype=float32, chunksize=(2560, 138), chunktype=numpy.ndarray>
<class 'dask.array.core.Array'>
flux_up_sw: dask.array<rechunk-merge, shape=(1085440, 138), dtype=float32, chunksize=(2560, 138), chunktype=numpy.ndarray>
<class 'dask.array.core.Array'>
flux_dn_lw: dask.ar

In [6]:
def broadcast_features(array: da.Array):
    a = da.repeat(array, 138, axis=-1)
    a = da.moveaxis(a, -2, -1)
    return a

def pad_tensor(array: da.Array):
    a = da.pad(array, ((0, 0), (1, 1), (0, 0)))
    return a

In [20]:
data

{'sca_inputs': dask.array<rechunk-merge, shape=(1085440, 17), dtype=float32, chunksize=(2560, 17), chunktype=numpy.ndarray>,
 'col_inputs': dask.array<rechunk-merge, shape=(1085440, 137, 27), dtype=float32, chunksize=(2560, 137, 27), chunktype=numpy.ndarray>,
 'hl_inputs': dask.array<rechunk-merge, shape=(1085440, 138, 2), dtype=float32, chunksize=(2560, 138, 2), chunktype=numpy.ndarray>,
 'inter_inputs': dask.array<rechunk-merge, shape=(1085440, 136, 1), dtype=float32, chunksize=(2560, 136, 1), chunktype=numpy.ndarray>,
 'flux_dn_sw': dask.array<rechunk-merge, shape=(1085440, 138), dtype=float32, chunksize=(2560, 138), chunktype=numpy.ndarray>,
 'flux_up_sw': dask.array<rechunk-merge, shape=(1085440, 138), dtype=float32, chunksize=(2560, 138), chunktype=numpy.ndarray>,
 'flux_dn_lw': dask.array<rechunk-merge, shape=(1085440, 138), dtype=float32, chunksize=(2560, 138), chunktype=numpy.ndarray>,
 'flux_up_lw': dask.array<rechunk-merge, shape=(1085440, 138), dtype=float32, chunksize=(256

In [21]:
from typing import Dict

# still lazy
@dask.delayed
def perform_feature_engineering(data: Dict[str, da.Array]):
    print(data)
    
    print(type(data['hl_inputs']))
    print("feature engineering x")
    x = da.concatenate([
        data['hl_inputs'],
        pad_tensor(data['inter_inputs']),
        broadcast_features(data['sca_inputs'][..., np.newaxis])
    ], axis=-1)
    
    print("feature engineering y")
    y = da.concatenate([
        data['flux_dn_sw'][..., np.newaxis],
        data['flux_up_sw'][..., np.newaxis],
        data['flux_dn_lw'][..., np.newaxis],
        data['flux_up_lw'][..., np.newaxis],
    ], axis=-1)

    print(f"x of shape: {x.shape}")
    print(f"y of shape: {y.shape}")
    
    return x, y

In [23]:
from pprint import pprint

print(data)

x, y = perform_feature_engineering(data).compute()
out_file = osp.join(processed_data_path, 'feats.h5')
x.to_hdf5(out_file, '/x')
y.to_hdf5(out_file, '/y')

{'sca_inputs': dask.array<rechunk-merge, shape=(1085440, 17), dtype=float32, chunksize=(2560, 17), chunktype=numpy.ndarray>, 'col_inputs': dask.array<rechunk-merge, shape=(1085440, 137, 27), dtype=float32, chunksize=(2560, 137, 27), chunktype=numpy.ndarray>, 'hl_inputs': dask.array<rechunk-merge, shape=(1085440, 138, 2), dtype=float32, chunksize=(2560, 138, 2), chunktype=numpy.ndarray>, 'inter_inputs': dask.array<rechunk-merge, shape=(1085440, 136, 1), dtype=float32, chunksize=(2560, 136, 1), chunktype=numpy.ndarray>, 'flux_dn_sw': dask.array<rechunk-merge, shape=(1085440, 138), dtype=float32, chunksize=(2560, 138), chunktype=numpy.ndarray>, 'flux_up_sw': dask.array<rechunk-merge, shape=(1085440, 138), dtype=float32, chunksize=(2560, 138), chunktype=numpy.ndarray>, 'flux_dn_lw': dask.array<rechunk-merge, shape=(1085440, 138), dtype=float32, chunksize=(2560, 138), chunktype=numpy.ndarray>, 'flux_up_lw': dask.array<rechunk-merge, shape=(1085440, 138), dtype=float32, chunksize=(2560, 138)

AttributeError: 'numpy.ndarray' object has no attribute 'chunks'

In [9]:
x.shape

NameError: name 'x' is not defined

In [9]:
dask.config.set(scheduler='processes')

out_dir = osp.join(processed_data_path, 'feats_npy')
da.to_npy_stack(osp.join(out_dir, 'x'), x, axis=0)
da.to_npy_stack(osp.join(out_dir, 'y'), y, axis=0)